In [1]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, Lambda
from tensorflow.keras.utils import to_categorical
import tensorflow
import pandas as pd
import numpy as np

In [2]:
normal_model = load_model("my_model.h5")
test = pd.read_csv('./input/train.csv')
print("Data are Ready!!")
print(f"Testing data size is {test.shape}")

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-08-10 21:29:39.955608: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-10 21:29:39.955731: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Data are Ready!!
Testing data size is (42000, 785)


In [3]:
X_test = test.drop(['label'], 1).values
y_test = test['label'].values

_, X_test, _, y_test = train_test_split(X_test, y_test, test_size=0.1, random_state=0)

X_test = X_test / 255.0
X_test = X_test.reshape(-1,28,28,1)

y_test = to_categorical(y_test)

print(f"Label size {y_test.shape}")

/var/folders/v3/ybc9hrrd0yg6d_zr2sbckrtm0000gn/T/ipykernel_5604/2536502158.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_test = test.drop(['label'], 1).values


Label size (4200, 10)


In [4]:
def random_multiplier(shape, mean=1.01, scale=0.5):
    random_factors = np.random.normal(loc=mean, scale=scale, size=shape)
    # random_factors = np.random.uniform(0.99, 1.01, shape)
    return random_factors

class RandomMultiplierLayer(tensorflow.keras.layers.Layer):
    def call(self, inputs):
        random_factors = random_multiplier(inputs.shape[-1])
        return inputs * random_factors

# 커스텀 레이어 추가하여 새 모델 구축
error_model = Sequential()
for layer in normal_model.layers:
    error_model.add(layer)  # 기존 레이어 추가
    # if isinstance(layer, Conv2D) or isinstance(layer, Dense):
    error_model.add(RandomMultiplierLayer())  # 랜덤한 값을 곱하는 레이어 추가

# 컴파일 및 요약
error_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
def simulate_gaussian_noise(model, mean=1.01, scale=0.5):
    for layer in model.layers:
        if isinstance(layer, (tensorflow.keras.layers.Conv2D, tensorflow.keras.layers.Dense)):
            weights = layer.get_weights()[0]
            biases = layer.get_weights()[1]

            noisy_weights = weights * np.random.normal(loc=mean, scale=scale, size=weights.shape)
            noisy_biases = biases * np.random.normal(loc=mean, scale=scale, size=biases.shape)
            # noisy_weights = weights * np.random.uniform(0.99, 1.01, weights.shape)
            # noisy_biases = biases * np.random.uniform(0.99, 1.01, biases.shape)
            # noisy_weights = weights * np.random.choice([-1, 2], size=weights.shape, p=[0.99,0.01])
            # noisy_biases = biases * np.random.choice([-1, 2], size=biases.shape, p=[0.99,0.01])
            print(weights, noisy_weights, sep='WWW')
            # print(np.random.choice([-1, 2], size=weights.shape, p=[0.99,0.01]))
            layer.set_weights([noisy_weights, noisy_biases])

simulate_gaussian_noise(error_model)
normal_model = load_model("my_model.h5")

[[[[-5.95359392e-02 -3.60312611e-02  8.33147094e-02 -7.00996965e-02
    -5.23091480e-02 -7.42706060e-02  6.34248704e-02 -9.38052833e-02
    -8.36731717e-02  3.04841273e-03 -7.07882270e-02  8.81479457e-02
     2.53956784e-02 -8.72810464e-03  1.33443521e-02 -5.17015010e-02
     9.84963775e-02 -6.59798980e-02 -2.60051321e-02  9.46237706e-03
    -2.71798950e-02 -2.34302208e-02 -1.03274658e-01  6.03190549e-02
    -6.49480075e-02 -1.43344000e-01 -9.11515057e-02 -6.40814304e-02
     5.73980138e-02 -8.66763815e-02 -1.94738973e-02 -2.14165747e-02
    -1.13555402e-01  3.01821679e-02 -8.13209414e-02 -8.32797065e-02
     7.54614547e-02 -9.06084850e-02 -2.00066101e-02 -3.94805335e-02
    -6.43486157e-02  8.86142999e-02  6.35844842e-02 -5.49779758e-02
    -1.81268543e-01  1.78758390e-02  1.39051797e-02  5.24065644e-02
    -2.59520747e-02  9.47436020e-02  1.20278478e-01  9.18042287e-02
     9.78992432e-02 -1.76050272e-02 -8.63536224e-02  4.54620793e-02
    -1.08760476e-01  1.24738812e-02 -3.09228208e

In [6]:
test_error_loss, test_error_accuracy = error_model.evaluate(X_test, y_test, verbose=2)
test_normal_loss, test_normal_accuracy = normal_model.evaluate(X_test, y_test, verbose=2)
print(f"Test normal model accuracy: {test_normal_accuracy} Test error model accuracy: {test_error_accuracy}")

2023-08-10 21:29:42.331059: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-10 21:29:42.534986: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 2s - loss: 6.1336 - accuracy: 0.4326 - 2s/epoch - 12ms/step


2023-08-10 21:29:43.980806: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 2s - loss: 0.0145 - accuracy: 0.9950 - 2s/epoch - 11ms/step
Test normal model accuracy: 0.9950000047683716 Test error model accuracy: 0.43261903524398804


In [7]:
def save_weights_to_text(model, file_path):
    with open(file_path, 'w') as file:
        for layer in model.layers:
            weights = layer.get_weights()
            if len(weights) > 0:
                weights_text = [str(w.tolist()) for w in weights]
                weights_str = "\n".join(weights_text)
                file.write(f"Layer: {layer.name}\n")
                file.write(weights_str + "\n\n")

In [8]:
weights_file_path = './model_weights.txt'
save_weights_to_text(normal_model, weights_file_path)

In [9]:
weights_file_path = './error_model_weights.txt'
save_weights_to_text(error_model, weights_file_path)